# LLM evaluation demo

First install dependences, download and unpack data

In [ ]:
!pip install datasets torch openai tiktoken nltk seaborn matplotlib 

In [ ]:
!git clone https://github.com/booydar/babilong source
!unzip source/data/tasks_1-20_v1-2.zip -d data/

In [ ]:
import openai 
import asyncio
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset
import numpy as np
import tiktoken
import time
import os
import datasets
from source.babilong_utils import TaskDataset, SentenceSampler, NoiseInjectionDataset

# Choose parameters for evaluation:
* task: babilong task (qa1 - qa10). Use different prompts for each task! You can find our prompts in the Appendix B of our paper
* task_path: path to the file with task data
* message_lengths: list of task lengths to evaluate. It must be less then the LLM's context window
* number_of_samples: number of instances of the task with message_length to evaluate

# Choose LLM
* model: ChatGPT model
    * 'gpt-4-1106-preview' - GPT4-Turbo, context length up to 128k
    * 'gpt-4' - GPT4, context length up to 32k
    * 'gpt-3.5-turbo-1106' - GPT3.5, context length up to 16k
    * 'ft:gpt-3.5-turbo-1106:personal:babilong-qa1:8nZsdRrH' - GPT3.5 fintunned for qa1 task, context length up to 16k
* token: your token for OpenAI api

In [ ]:
task = 'qa1'
task_path = 'data/tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_test.txt'
message_lengths = [4000, 8000, 16000, 32000, 64000, 128000]
number_of_samples = 25

model = 'gpt-4-1106-preview'
token = 'YOUR_CHAT_GPT_TOKEN'

# Adapt tiktoken api to babilong utils

In [ ]:
class Tokenizer():
    def __init__(self, model):
        self.impl_ = tiktoken.encoding_for_model(model)

    def __call__(self, inp):
        if isinstance(inp, list):
            result = self.impl_.encode_batch(inp)
        else:
            result = self.impl_.encode(inp)
        return {
            'input_ids': result
        }

    def encode(self, inp, add_special_tokens):
        return self.impl_.encode(inp)

    def decode(self, inp):
        return self.impl_.decode(inp)

    def decode_batch(self, inp):
        return self.impl_.decode_batch(inp)

# Choose noise dataset

In this example we use wikitext to generate noise. 
To reproduce results from our paper use PG19.

In [ ]:
noise_dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')
#noise_dataset = datasets.load_from_disk("pg19-data-test")

# Run evaluation loop and save results in .csv format

In [ ]:
# create directory to save LLM responses
os.makedirs(task, exist_ok=True)

client = openai.OpenAI(api_key=token)

tokenizer = Tokenizer(model)

task_dataset_test = TaskDataset(task_path)
noise_sampler_test = SentenceSampler(noise_dataset['test'], tokenizer=tokenizer)

for message_length in message_lengths:
    outfile =  f'{task}/msg_{message_length}.csv'
    df = pd.DataFrame({
        'answer': [],
        'gpt4answer': [],
        'result': [],
    })

    # create dataset with given message length
    dataset_test = NoiseInjectionDataset(task_dataset=task_dataset_test,
                                         noise_sampler=noise_sampler_test,
                                         tokenizer=tokenizer,
                                         sample_size=message_length)
    
    # collect GPT responses and save them in .csv
    for i, sample in zip(range(number_of_samples), dataset_test):
        facts = sample['facts']
        question = sample['question']
        true_answer = tokenizer.decode(sample['target_tokens'])
        background_text = tokenizer.decode_batch(sample['background_text'])
        query = tokenizer.decode(sample['input_tokens'])
        
        messages = [ 
            {
                "role": "system",
                "content": "You are a intelligent assistant."
            },
            {
                "role": "user", 
                "content": 
                    "I give you context with the facts about positions of different persons hidden in some random text and a question. "
                    "You need to answer the question based only on the information from the facts. "
                    "If a person was in different locations use the latest location to answer the question.\n\n"
                    "<example>\n"
                    "Charlie went to the hallway. Judith come back to the kitchen. Charlie travelled to balcony. Where is Charlie?\n"
                    "Assistant: balcony\n"
                    "</example>\n\n"
                    "<example>\n"
                    "Alan moved to the garage. Charlie went to the beach. Alan went to the shop. Rouse travelled to balcony. Where is Alan?\n"
                    "Assistant: shop\n"
                    "</example>\n\n"
                    "<context>\n"
                    f"{query}"
                    "</context>\n\n"
                    f"QUESTION: {question}\n" 
                    "Your answer should be a single word - the most recent location of ’person’ in the question. "
                    "Do not write anything afer that."
            },
        ]
    
        response = client.chat.completions.create(model=model, messages=messages)
        gpt_answer = response.choices[0].message.content.strip().lower()
    
        if gpt_answer.endswith('.'):
            gpt_answer = gpt_answer[:-1]
    
        print(message_length, i, true_answer, gpt_answer)
    
        df.loc[len(df)] = [true_answer, gpt_answer, true_answer == gpt_answer]
        df.to_csv(outfile)

# Plot evaluation results

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pylab as plt
from matplotlib.colors import LinearSegmentedColormap

accuracy = np.zeros((1, len(message_lengths)))

for i, msg in enumerate(message_lengths):
    fname = f'{task}/msg_{msg}.csv'
    if not os.path.isfile(fname):
        print('not such file', fname)
        continue     
    df = pd.read_csv(fname, index_col=[0])
    last_word = df['gpt4answer'].apply(lambda x: x.split(' ')[-1]).apply(lambda x: x.split('\n')[-1])
    score = (last_word == df['answer']).sum()
    accuracy[0, i] = score / len(df)

font = {
    'size'   : 30
}

matplotlib.rc('font', **font)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 2)


cmap = LinearSegmentedColormap.from_list('ryg', ["red", "yellow", "green"], N=256)
ax = sns.heatmap(accuracy, cmap=cmap, vmin=0, vmax=1, annot=True, linewidth=0.5)
ax.set_xlabel('Context size')
labels = [f'{ln}' if ln < 1000 else f'{int(ln / 1000)}k' for ln in message_lengths]
ax.set_xticks(np.array(range(accuracy.shape[1])) + 0.5, labels)
ax.tick_params(axis='y', rotation=90)
ax.set_yticks([0], [''])
ax.set_title('Performance of '+f'{model}'+' on BABILong benchmark task '+f'{task}'+', accuracy')
plt.show()

fig.savefig('fig.pdf', dpi=100)